# Доверительные интервалы для среднего


### Задача 1
Пусть X ∼ F(x) — случайная величина с произвольным распределением, неизвестным математическим ожиданием 𝔼X и известной дисперсией 𝔻X. Как выглядит доверительный интервал для 𝔼X с приближённым уровнем доверия 100(1−α)%?

**Ответ**: $$ 𝔼X \pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{𝔻X}{n}}$$

### Задача 2
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

**Будьте осторожны при использовании метода std()!** Дело в том, что у объекта numpy он по умолчанию вычисляется как $\sqrt{\frac1{n}\sum_{i=1}^n\left(X_i-\bar{X}\right)^2}$,
а у объекта pandas — как $\sqrt{\frac1{n-1}\sum_{i=1}^n\left(X_i-\bar{X}\right)^2}$.

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на n-1.

In [11]:
import pandas as pd
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
import math

In [5]:
df = pd.read_csv('water.txt', sep = '\t')
df.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [12]:
mortality_mean = df.mortality.mean()
mortality_std = df.mortality.std(ddof = 1)/math.sqrt(len(df.mortality))

mortality_mean, mortality_std

(1524.1475409836066, 24.028521714278174)

In [13]:
map(lambda x: round(x, 4), _tconfint_generic(mortality_mean, mortality_std, len(df.mortality) - 1, 0.05, 'two-sided'))

[1476.0833, 1572.2117]

In [17]:
def get_confints(series, alpha = 0.05):
    series_mean = series.mean()
    series_std = series.std(ddof = 1)/math.sqrt(len(series))
    confint  = _tconfint_generic(series_mean, series_std, len(series) - 1, alpha, 'two-sided')
    return confint

### Вопрос 3
На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [19]:
get_confints(df[df.location == 'South'].mortality)

(1320.1517462936238, 1433.463638321761)

### Вопрос 4

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

 * Интервалы не пересекаются; видимо, средняя смертность на севере и на юге существенно разная
 * ~~Интервалы пересекаются; похоже, средняя смертность на севере и на юге одинаковая~~
 * ~~Интервалы пересекаются; видимо, средняя смертность на севере и на юге существенно разная~~
 * ~~Интервалы не пересекаются; похоже, средняя смертность на севере и на юге одинаковая~~

In [20]:
get_confints(df[df.location == 'North'].mortality)

(1586.5605251961385, 1680.6394748038613)

### Вопрос 5
Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [23]:
get_confints(df[df.location == 'South'].hardness)

(53.467198692036106, 86.071262846425441)

In [22]:
get_confints(df[df.location == 'North'].hardness)

(21.422487285724259, 39.377512714275738)

### Вопрос 6
Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^{2}$:
$$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$
При σ=1 какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью ±0.1?

**Решение**:
$$z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}} = 0.1$$

$$\frac{z_{1-\frac{\alpha}{2}}\sigma}{0.1} = \sqrt{n}$$

$$ n = (\frac{z_{1-\frac{\alpha}{2}}\sigma}{0.1})^{2} = 100z_{1-\frac{\alpha}{2}}^{2} \approx 385$$

In [26]:
from scipy.stats import norm

In [32]:
z = norm.ppf(0.975)
100*z**2

384.14588206941255

### Вопрос 7
Объём выборки, необходимый для построения доверительного интервала заданной ширины:

 * увеличивается с уменьшением требуемой ширины
 * ~~уменьшается с ростом дисперсии выборки~~
 * ~~увеличивается с увеличением требуемой ширины~~
 * ~~увеличивается с ростом α~~
 * увеличивается с ростом дисперсии выборки
 * уменьшается с ростом α